In [1]:
%load_ext autoreload
%autoreload 1
%matplotlib inline

In [2]:
import os, sys
from os.path import join as pjoin
sys.path.insert(0, pjoin(os.path.pardir, 'webapi'))
sys.path.insert(0, pjoin(os.path.pardir, os.path.pardir, 'mcdose'))
import numpy as np
import h5py
from bson import ObjectId
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt

os.environ['CUDA_VISIBLE_DEVICES'] = ""
%aimport loaders
%aimport mcdose
from mcdose.visualize import vis_slice
import database
from api_enums import DBCOLLECTIONS

In [3]:
# connect to database
database.init_dbclient(port=27099, auth=['root', 'rootpass'])
database.InitDataStorage(root='/media/hdd2/g4sim/data')

In [4]:
# get list of simdocs
beam_id = '5e15407ec9c073745a01ced3'
filter = {'_id': ObjectId(beam_id)}
beamdoc = database.db[DBCOLLECTIONS.BEAMPHOTON].find_one(filter)

# get sim_ids
sim_ids = []
filter = {'beam_id': ObjectId(beam_id), 'num_particles': 2000, 'magnetic_field.2': 1.5}
for simdoc in database.db[DBCOLLECTIONS.SIMULATION].find(filter):
    sim_ids.append(simdoc['_id'])
print(len(sim_ids))

285


In [5]:
simdoc = database.db[DBCOLLECTIONS.SIMULATION].find_one({'_id': sim_ids[140]})

# Test Model Prediction

In [6]:
from mcdose import get_trained_model
import json
import logging
mcdoselogger = logging.getLogger('MCDose')
mcdoselogger.addHandler(logging.StreamHandler())
mcdoselogger.setLevel(logging.DEBUG)
stats_file = '/media/hdd1/g4sim/runs_shenggpu2/0008/normstats.json'

model = get_trained_model(
    config='/media/hdd1/g4sim/runs_shenggpu2/0008/config.yml',
    weights='/media/hdd1/g4sim/runs_shenggpu2/0008/checkpoints/weights.hdf5',
    normstats=stats_file,
)
with open(stats_file, 'r') as fd:
    normstats = json.load(fd)

Config loaded from "/media/hdd1/g4sim/runs_shenggpu2/0008/config.yml"
Loaded model from "/media/hdd1/g4sim/runs_shenggpu2/0008/checkpoints/weights.hdf5"


In [7]:
def predict(inputs):
    assert(inputs.ndim == 4) # shouldn't have sample axis (0) yet
    factors = np.array(normstats['factor'][1:])[None, None, None, ...]
    inputs = np.multiply(inputs, factors)
    preds = model(inputs.astype(np.float32)[None, ...])
    return np.divide(preds[0, ..., 0], factors[...,0]) # prune sample/channel axes

#inputs = np.ones((1, 3, 3, 2))
arrayloader_orig = loaders.ArrayLoader(get_geom=True, get_label=True)
arrayloader = loaders.ArrayLoader(reorient=True, context=(12,55,12), get_geom=True, get_label=True, reversible=True)

from tqdm import tqdm
orig_beam_predicts = None
orig_beam_label = None
for sim_id in tqdm(sim_ids[120:140]):
    simdoc = database.db[DBCOLLECTIONS.SIMULATION].find_one({'_id': sim_id})
    result = arrayloader(simdoc)
    ((inputs,), geom, label), reverser = result
    inputs = np.stack([inputs, geom], axis=-1)

    predicts = predict(inputs)
    #vis_slice(geom[geom.shape[0]//2], predicts[predicts.shape[0]//2])
    #plt.show()
    orig_predicts, orig_geom, orig_label = reverser(predicts), reverser(geom), reverser(label)
    if orig_beam_predicts is None:
        orig_beam_predicts = orig_predicts
    else:
        orig_beam_predicts += orig_predicts
    if orig_beam_label is None:
        orig_beam_label = orig_label
    else:
        orig_beam_label += orig_label
    
    


  0%|          | 0/20 [00:02<?, ?it/s]


InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [1,25,56,14,20] vs. shape[1] = [1,25,56,13,20] [Op:ConcatV2] name: concat

In [ ]:

slicenum = 51
simdoc = database.db[DBCOLLECTIONS.SIMULATION].find_one({'_id': sim_ids[0]})
(_, orig_geom_full, _)  = arrayloader_orig(simdoc)

plt.figure()
vis_slice(orig_geom_full[slicenum], orig_beam_predicts[slicenum], thresh=0)
plt.title('Predictions')
plt.show()

plt.figure()
vis_slice(orig_geom_full[slicenum], orig_beam_label[slicenum], thresh=0)
plt.title('Labels')
plt.show()

plt.figure()
aims = vis_slice(orig_geom_full[slicenum], orig_beam_label[slicenum]-orig_beam_predicts[slicenum], thresh=0)
plt.title('label-prediction')
plt.colorbar(aims[1][1])
plt.show()
print(np.max(orig_beam_label[slicenum]), np.max(orig_beam_predicts[slicenum]))
